In [1]:
#？？？提取vs ph gr共有的点？？？
#这些数据首先用于生成高斯图，然后输入到CNN中以预测1-D Vs
#内容输出到./Output/disp_pg_real文件中

#导入库
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import os

#给出步骤1，2提取的相速度和群速度数据的路径
filepath_ph = './Output/dispersion_ph/'
filepath_gr = './Output/dispersion_gr/'

#创建disp_combine_gaussian_map_test、Figs和disp_pg_real文件夹
#???  高斯图像是什么？高斯图像的目的是什么？如何生成高斯图像？
os.system('mkdir ./Output/disp_combine_gaussian_map_test')
os.system('mkdir Figs')
os.system('mkdir ./Output/disp_pg_real')

#创建数组：深度depth、depth_mark和periods
#depth = np.array([2.5,5,10,15,20,30,40,60,80,100,120,150,180])
depth_mark = np.array([0,5,10,15,20,30,40,60,80,100,120,150,180])
depth = np.arange(0,150.5,0.5)
#???  8-50是什么意思？？？秒数吗？
periods = np.array([8,10,12,14,16,18,20,22,24,26,28,30,32,35,40,45,50])
count = 0

#???  纬度为什么不从4度开始？
for lat in np.arange(4,60,0.5):
    for lon in np.arange(70,140,0.5):
        key = '%.2f'%lat + '_' + '%.2f'%lon
        key_name = key + '.txt'
        file_gr = filepath_gr + key_name
        file_ph = filepath_ph + key_name
        if os.path.exists(file_ph) and os.path.exists(file_gr):
            temp = np.loadtxt(file_ph); freqs_p = temp[:,0]; vel_p = temp[:,1];error_p = temp[:,2]
            temp = np.loadtxt(file_gr); freqs_g = temp[:,0]; vel_g = temp[:,1];error_g = temp[:,2]
            if len(temp) > 1:
                count = count + 1
                #用这些数据去预测CNN生成的Vs曲线
                tpg_out = np.array([periods,vel_p,error_p,vel_g,error_g])
                np.savetxt('./Output/disp_pg_real/' + '%.2f'%lat + '_' + '%.2f'%lon + '.txt',tpg_out,fmt = '%10.5f')
print(count)

4518


In [2]:
#???为什么要使用高斯映射？
#vel_axis矩阵的行数是rows，vel矩阵的行数是cols，生成vel_map行数和列数个0
def gaussian_map(vel,vel_axis,radius = 0.1):
    rows = vel_axis.shape[0]
    cols = vel.shape[0]
    vel_map = np.zeros((rows,cols))
    #range(cols)结果是0，1，2，……,cols-1
    for i in range(cols):
        vel_temp = vel[i]
        x_gaussian = gaussian(vel_temp, vel_axis, r=radius)
        vel_map[:,i] = x_gaussian
    return vel_map
def gaussian(vel,vel_axis,r=0.1):
    x_gaussian = np.exp(-(vel_axis-vel)**2/2*r**2)
    return x_gaussian

In [10]:
#从1到5生成60个数,包含1和5
vel_axis = np.linspace(1,5,num=60);r = 0.1
#???plot_flag是干什么用的？
plot_flag = False;count = 0
fig = plt.figure(1,figsize = (10,4))
for lat in np.arange(4,60,0.5):
    for lon in np.arange(70,140,0.5):
        key = "%.2f"%lat + "_" + "%.2f"%lon
        key_name = key + '.txt'
        file_gr = filepath_gr + key_name
        file_ph = filepath_ph + key_name
        if os.path.exists(file_gr) and os.path.exists(file_ph):
            temp = np.loadtxt(file_ph);freqs_ph = temp[:,0];vel_p = temp[:,1];error_p = temp[:,2]
            temp = np.loadtxt(file_gr);freqs_gr = temp[:,0];vel_g = temp[:,1];error_g = temp[:,2]
            if len(temp) > 1:
                count = count + 1
                #vel_map_p &vel_map_g是经过高斯函数运算出来的 vel_p列、vel_axis行的？？？数据？？？？
                vel_map_p = gaussian_map(vel_p,vel_axis,radius=r)
                vel_map_g = gaussian_map(vel_g,vel_axis,radius=r)
                name_pg = './Output/disp_combine_gaussian_map_test/c_' + '%.2f'%lat + '_' + '%.2f'%lon +'.npy'
                np.save(name_pg,np.array([vel_map_p,vel_map_g]))
                if not(plot_flag):
                    #subplot把图分成了1行2列，图像创建在第1个区域
                    plt.subplot(1,2,1)
                    #imshow接收一张图像，只是画出该图，但不会显示,aspect是纵横比、origin显示索引的位置、
                    #extent坐标位置
                    plt.imshow(vel_map_p,aspect = 'auto',origin= 'lower',extent = [8,50,2,5])
                    plt.xlabel('T(s)')
                    plt.ylabel('Vs(km/s)')
                    plt.title('Phase Velocity')
                    plt.subplot(1,2,2)
                    plt.imshow(vel_map_g,aspect = 'auto',origin = 'lower',extent = [8,50,2,5])
                    plt.xlabel('T(s)')
                    plt.ylabel('Vs(km/s)')
                    plt.title('Group Velocity')
                    #plt.tight_layout() 自动调整子图参数以提供指定的填充
                    plt.tight_layout()
                    plt.savefig('./Figs/'+ key +'.jpg')
                    #plt.show()
                    fig.clear()

<Figure size 720x288 with 0 Axes>